In [2]:
# ============================================
# 0) 설치
# ============================================
!pip -q install transformers sentencepiece accelerate datasets \
             sentence-transformers faiss-cpu pypdf gradio

# (선택) 스캔 PDF(OCR 필요)일 때:
# !apt -y install poppler-utils tesseract-ocr
# !pip -q install pdf2image pytesseract



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 24.5 MB/s eta 0:00:00


In [3]:
# ============================================
# 1) 라이브러리 & 환경
# ============================================
import os, json, re, math
from typing import List, Dict, Any, Tuple
import numpy as np

import torch
DEVICE = 0 if torch.cuda.is_available() else -1

from pypdf import PdfReader

from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer
import faiss

import gradio as gr
from google.colab import files


In [4]:
# ============================================
# 2) PDF 업로드 & 텍스트 추출
# ============================================
print("PDF 파일을 업로드하세요 (제품 사용설명서).")
uploaded = files.upload()

pdf_path = list(uploaded.keys())[0]  # 첫 번째 업로드 파일

def extract_text_from_pdf(pdf_file: str) -> List[Dict[str, Any]]:
    """
    각 페이지의 텍스트를 추출해 [{page, text}] 리스트로 반환
    """
    reader = PdfReader(pdf_file)
    pages = []
    for i, page in enumerate(reader.pages, start=1):
        try:
            text = page.extract_text() or ""
        except Exception:
            text = ""
        # 공백 정리
        text = re.sub(r'\s+', ' ', text).strip()
        pages.append({"page": i, "text": text})
    return pages

pages = extract_text_from_pdf(pdf_path)
total_chars = sum(len(p["text"]) for p in pages)
print(f"총 {len(pages)} 페이지, 약 {total_chars:,} 글자 텍스트를 추출했습니다.")


PDF 파일을 업로드하세요 (제품 사용설명서).


Saving LX3_2026_ko_KR.pdf to LX3_2026_ko_KR.pdf
총 590 페이지, 약 461,888 글자 텍스트를 추출했습니다.


In [5]:
# ============================================
# 3) 텍스트 청크화 (슬라이딩 윈도우)
# ============================================
# 너무 긴 문서 → 고정 길이(문자수) 청크로 나누고, 페이지 정보 보존
CHUNK_SIZE = 900   # 문자 단위 권장 700~1200 사이
CHUNK_OVERLAP = 150

def chunk_page_text(page_text: str, page_no: int) -> List[Dict[str, Any]]:
    chunks = []
    text = page_text
    n = len(text)
    if n == 0:
        return chunks
    start = 0
    while start < n:
        end = min(start + CHUNK_SIZE, n)
        chunk = text[start:end]
        chunks.append({
            "page": page_no,
            "text": chunk
        })
        if end == n:
            break
        start = end - CHUNK_OVERLAP
        start = max(start, 0)
    return chunks

all_chunks: List[Dict[str, Any]] = []
for p in pages:
    all_chunks.extend(chunk_page_text(p["text"], p["page"]))

print(f"청크 개수: {len(all_chunks)} (청크 길이 ~{CHUNK_SIZE}, 중첩 {CHUNK_OVERLAP})")


청크 개수: 822 (청크 길이 ~900, 중첩 150)


In [6]:
# ============================================
# 4) 임베딩(문장 벡터) & FAISS 인덱스 구축
# ============================================
# 한국어/다국어 메뉴얼 모두 커버 가능한 소형 멀티링구얼 모델
EMB_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
emb_model = SentenceTransformer(EMB_MODEL_NAME)

def build_faiss_index(chunks: List[Dict[str, Any]]):
    texts = [c["text"] for c in chunks]
    vecs = emb_model.encode(texts, normalize_embeddings=True, batch_size=64, show_progress_bar=True)
    vecs = np.array(vecs, dtype="float32")
    index = faiss.IndexFlatIP(vecs.shape[1])   # cosine 유사도 = dot (normalize 전제)
    index.add(vecs)
    return index, vecs

faiss_index, chunk_vecs = build_faiss_index(all_chunks)
print("FAISS 인덱스 구축 완료.")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

FAISS 인덱스 구축 완료.


In [7]:
# ============================================
# 5) QA 파이프라인 준비 (한국어 우선 → 다국어 대체)
# ============================================
def load_qa_pipeline():
    # 1순위: 한국어 KorQuAD 파인튜닝
    try:
        qa = pipeline(
            "question-answering",
            model="monologg/koelectra-small-v3-finetuned-korquad",
            tokenizer="monologg/koelectra-small-v3-finetuned-korquad",
            device=DEVICE
        )
        return qa, "koelectra-korquad"
    except Exception as e:
        print("한국어 QA 모델 로드 실패, 다국어로 대체:", e)
    # 2순위: xlm-roberta (다국어 SQuAD2)
    qa = pipeline(
        "question-answering",
        model="deepset/xlm-roberta-base-squad2",
        tokenizer="deepset/xlm-roberta-base-squad2",
        device=DEVICE
    )
    return qa, "xlmr-squad2"

qa_pipe, qa_model_name = load_qa_pipeline()
print(f"QA 모델: {qa_model_name}")


config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/56.3M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/56.3M [00:00<?, ?B/s]

Device set to use cuda:0


QA 모델: koelectra-korquad


In [8]:
# ============================================
# 6) 검색 → 상위 K 청크에서 Extractive QA 실행
# ============================================
TOP_K = 5            # 검색 상위 청크 수
MAX_CONTEXT_CHARS = 1800  # QA 컨텍스트 길이를 제한(너무 길면 성능 저하)

def retrieve(query: str, k: int = TOP_K) -> List[Tuple[int, float]]:
    qvec = emb_model.encode([query], normalize_embeddings=True)[0].astype("float32")
    D, I = faiss_index.search(qvec.reshape(1, -1), k)
    # 반환: (청크 인덱스, 유사도)
    return list(zip(I[0].tolist(), D[0].tolist()))

def make_context(top_hits: List[Tuple[int,float]]) -> Tuple[str, List[int]]:
    """
    상위 청크들을 이어붙여 QA 컨텍스트 생성 (길이 제한)
    반환: (context_text, source_pages)
    """
    ctx_list, pages, acc_len = [], [], 0
    for idx, score in top_hits:
        ch = all_chunks[idx]
        t = ch["text"]
        if acc_len + len(t) > MAX_CONTEXT_CHARS:
            # 조금만 추가해서 자르기
            remain = MAX_CONTEXT_CHARS - acc_len
            if remain > 200:  # 최소 200자 이상일 때만 덧붙임
                t = t[:remain]
                ctx_list.append(t)
                pages.append(ch["page"])
            break
        ctx_list.append(t)
        pages.append(ch["page"])
        acc_len += len(t)
    context = "\n".join(ctx_list)
    return context, sorted(list(set(pages)))

def answer_question(question: str) -> Dict[str, Any]:
    hits = retrieve(question, k=TOP_K)
    context, src_pages = make_context(hits)
    if len(context.strip()) < 30:
        return {
            "answer": "설명서에서 관련 내용을 찾지 못했습니다. 질문을 더 구체적으로 작성해 주세요.",
            "score": 0.0,
            "pages": [],
            "snippets": []
        }
    # 여러 청크 각각에서 QA를 돌려 최고 점수 선택(대안: 통합 컨텍스트 1회 실행)
    # 여기선 '통합 컨텍스트 1회 실행'으로 간단화
    res = qa_pipe({"question": question, "context": context})
    # 스니펫(정답 주변 문장) 추출
    start, end = res.get("start", 0), res.get("end", 0)
    snippet_start = max(0, start-120)
    snippet_end   = min(len(context), end+120)
    snippet = context[snippet_start:snippet_end]

    return {
        "answer": res.get("answer", "").strip(),
        "score": float(res.get("score", 0)),
        "pages": src_pages,
        "snippets": [snippet]
    }

# 짧은 테스트
print(answer_question("초기 설정 방법은 무엇인가요?"))


/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


{'answer': '인포테인먼트 시 스템 웹 매뉴얼을', 'score': 0.3564507730770856, 'pages': [160, 221, 232, 285], 'snippets': [' 빌트인 캠 메뉴 내 주행/주차 녹화 설정 및 충격 감지 민감도 공조 시스템 다음 공조 기능들의 최근 작동 상태 온도,\nӝ 업데이트로 인해 인포테인먼트 시스템의 상세 설정이 변경될 수 있습니다. 자세한 설정 방법은 인포테인먼트 시 스템 웹 매뉴얼을 참고하십시오. 편의 장치224\n기타 기능 설정하기 인포테인먼트 시스템에서 에어컨 자동 건조(애프터 블로우), 자동 김서림 제거(오토 디포그), 습기 발생 저감, 와셔액 냄새 유입 방지 등 다양한 부가 기능을 설정할']}


In [9]:
# ============================================
# 7) (선택) 섹션 요약 파이프라인
# ============================================
from transformers import AutoModelForSeq2SeqLM

def load_summarizer():
    try:
        # 한국어 요약 (KoBART)
        smz = pipeline("summarization", model="gogamza/kobart-summarization", device=DEVICE)
        name = "kobart-summarization"
    except Exception as e:
        print("한국어 요약 모델 로드 실패, 다국어 요약으로 대체:", e)
        smz = pipeline("summarization", model="facebook/bart-large-cnn", device=DEVICE)
        name = "bart-cnn"
    return smz, name

summarizer, smz_name = load_summarizer()
print(f"Summarizer: {smz_name}")

def summarize_topic(keyword: str, k: int = 4, max_chars: int = 2000) -> str:
    """
    키워드로 관련 청크를 모아 압축 요약
    """
    hits = retrieve(keyword, k=k)
    ctx, pages = make_context(hits)
    ctx = ctx[:max_chars]
    if len(ctx) < 120:
        return "요약할 텍스트를 충분히 찾지 못했습니다."
    out = summarizer(ctx, max_length=220, min_length=60, do_sample=False)[0]["summary_text"]
    return f"[관련 페이지: {pages}] {out}"


config.json: 0.00B [00:00, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
Device set to use cuda:0


Summarizer: kobart-summarization


In [10]:
# ============================================
# 8) Gradio 간단 챗봇 UI
# ============================================
with gr.Blocks(title="제품 사용설명서 QA") as demo:
    gr.Markdown("## 📘 제품 사용설명서 QA 챗봇\nPDF로부터 구축된 임베딩+QA로 답변합니다. 질문을 입력하세요.")
    with gr.Row():
        q = gr.Textbox(label="질문", placeholder="예) Wi-Fi 초기 설정 방법은?")
    with gr.Row():
        btn = gr.Button("질문하기")
        smz_btn = gr.Button("요약(키워드)")
        kw = gr.Textbox(label="요약 키워드(선택)", placeholder="예) 초기 설정, 보증, 안전 주의")
    ans = gr.Textbox(label="답변", lines=4)
    meta = gr.Textbox(label="참고 페이지 / 스니펫", lines=6)

    def on_ask(question):
        res = answer_question(question)
        pages = ", ".join(map(str, res["pages"]))
        snippets = "\n---\n".join(res["snippets"])
        meta_text = f"[참고 페이지] {pages}\n[신뢰도] {res['score']:.3f}\n[스니펫]\n{snippets}"
        return res["answer"], meta_text

    def on_summarize(keyword):
        return summarize_topic(keyword), ""

    btn.click(on_ask, inputs=[q], outputs=[ans, meta])
    smz_btn.click(on_summarize, inputs=[kw], outputs=[ans, meta])

demo.launch(debug=True, share=False)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

/usr/local/lib/python3.12/dist-packages/transformers/pipelines/question_answering.py:395: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
Both `max_new_tokens` (=256) and `max_length`(=220) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=220) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Keyboard interruption in main thread... closing server.
